In [ ]:
!pip install ujson

In [ ]:
import random
import os
import gzip
import bz2
import csv
import ujson as json
import glob
import math

In [ ]:
def write_file(out_file,mkdir=True,binary=False):
  if mkdir:
    dir = os.path.split(out_file)[0]
    if dir:
      os.makedirs(dir,exist_ok=True)

  if binary:
    if out_file.endswith('.gz'):
      return gzip.open(out_file,'wb')
    elif out_file.endswith('.bz2'):
      return bz2.open(out_file,'wb')
    else:
      return open(out_file,'wb')

  else:
    if out_file.endswith('.gz'):
      return gzip.open(out_file,'wt',encoding='utf-8')
    elif out_file.endswith('.bz2'):
      return bz2.open(out_file,'wt',encoding='utf-8')
    else:
      return open(out_file,'w',encoding='utf-8')




In [ ]:
def read_file(in_file,binary=False,errors=None):

  if binary:
    if in_file.endswith('.gz'):
      return gzip.open(in_file,'rb')
    elif in_file.endswith('.bz2'):
      return bz2.open(in_file,'rb')
    else:
      return open(in_file,'rb')

  else:
    if in_file.endswith('.gz'):
      return gzip.open(in_file,'rt',encoding='utf-8',errors=errors)
    elif in_file.endswith('.bz2'):
      return bz2.open(in_file,'rt',encoding='utf-8',errors=errors)
    else:
      return open(in_file,'r',encoding='utf-8',errors=errors)


In [ ]:
def shuffle_blocks(it,*,block_size=20000,rand=random):
  assert block_size>=4
  block = []
  for i in it:
    block.append(i)
    if len(block)>=block_size:
      rand.shuffle(block)
      for _ in range(block_size//2):
        yield block.pop(-1)

  rand.shuffle(block)
  for b in block:
    yield b

In [ ]:
def expand_file(input,file_pattern='*',files=None):
  if type(input) is str:
    if ':' in input:
      input = input.split(':')
    else:
      input = [input]

  all_inputs = []
  if files is None:
    files = []

  for i in input:
    print(i)
    if i in files:
      continue
    if os.path.isdir(i):
      sub_file = glob.glob(i+"/**/"+file_pattern,recursive=True)
      sub_file = [f for f in sub_file if not os.path.isdir(f)]
      sub_file = [f for f in sub_file if f not in input and f not in files]
      all_inputs.extend(sub_file)
    else:
      all_inputs.append(i)

  all_inputs.sort()
  return all_inputs



In [ ]:
def j_lines(input,files=None,limit=0,report_every=100000,*,errors=None,shuffled=None):
  lst = [f for f in expand_file(input,"*.jsonl*",files) if not f.endswith('.lock')]

  return read_lines(lst,limit=limit,report_every=report_every,errors=errors,shuffled=shuffled)


In [ ]:
def read_lines(input,limit=0,report_every=100000,*,errors=None,shuffled=None):
  count =0
  input = expand_file(input)
  if shuffled:
    if type(shuffled) != random.Random:
      shuffled = random.Random()

    open_blocks = int(math.ceil(len(input)/32.0))
    for open_i in range(open_blocks):
      open_files = [read_file(i,errors=errors) for i in input[open_i::open_blocks]]
      while len(open_files)>0:
        fx = shuffled.randint(0,len(open_files)-1)
        next_l = open_files[fx].readline()
        if next_l:
          yield next_l
          count +=1

        else:
          open_files[fx].close()
          del open_files[fx]

  else:
    for i in input:
      with read_file(i,errors=errors) as fp:
        for l in fp:
          yield l
          count +=1
          if 0<limit<= count:
            return



## Tabmcq

In [ ]:
!wget https://ai2-public-datasets.s3.amazonaws.com/tablestore-questions/TabMCQ_v_1.0.zip
!unzip TabMCQ_v_1.0.zip

--2023-11-18 14:23:43--  https://ai2-public-datasets.s3.amazonaws.com/tablestore-questions/TabMCQ_v_1.0.zip
Resolving ai2-public-datasets.s3.amazonaws.com (ai2-public-datasets.s3.amazonaws.com)... 52.92.179.177, 52.218.133.217, 52.92.181.33, ...
Connecting to ai2-public-datasets.s3.amazonaws.com (ai2-public-datasets.s3.amazonaws.com)|52.92.179.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 698796 (682K) [binary/octet-stream]
Saving to: ‘TabMCQ_v_1.0.zip’

TabMCQ_v_1.0.zip    100%[===================>] 682.42K  3.27MB/s    in 0.2s    

2023-11-18 14:23:43 (3.27 MB/s) - ‘TabMCQ_v_1.0.zip’ saved [698796/698796]

Archive:  TabMCQ_v_1.0.zip
  inflating: MCQs.tsv                
  inflating: README.txt              
  inflating: TableIndex.xlsx         
   creating: Tables/
  inflating: Tables/.DS_Store        
   creating: Tables/auto/
  inflating: Tables/auto/18.tsv      
  inflating: Tables/auto/fb1.tsv     
  inflating: Tables/auto/fb2.tsv     
  inflating:

In [ ]:
rand = random.Random(1234)

In [ ]:
tid2rows = dict()
path = "./row_docs.jsonl"
row_search_out = write_file(path)

In [ ]:
for subdir in ['auto','monarch','regents']:
  subdir_f = os.path.join('./Tables',subdir)
  for file in os.listdir(subdir_f):
    # assert file[-4:] == '.tsv'
    # print(file)
    table_id = subdir+'-'+file[:-4]
    # print(table_id)
    rows = []
    with read_file(os.path.join(subdir_f,file)) as fp:
      for r,part in enumerate(csv.reader(fp,doublequote=False,delimiter='\t')):
        row = [c.strip() for c in part]
        # print(r,row,part)
        rows.append(row)
    tid2rows[table_id] = rows









Streaming output truncated to the last 5000 lines.
270 ['the main purpose of a meter stick', 'measure', 'length of an object'] ['the main purpose of a meter stick', 'measure', 'length of an object']
271 ['the use of spring balances', 'measure', 'the mass of objects'] ['the use of spring balances', 'measure', 'the mass of objects']
272 ['other decomposer bacteria', 'metabolize', 'inorganic solids'] ['other decomposer bacteria', 'metabolize', 'inorganic solids']
273 ['all animals', 'need', 'air food'] ['all animals', 'need', 'air food']
274 ['all living organisms', 'need', 'a proper habitat'] ['all living organisms', 'need', 'a proper habitat']
275 ['all organisms', 'need', 'a source of energy'] ['all organisms', 'need', 'a source of energy']
276 ['all plants', 'need', 'sunlight'] ['all plants', 'need', 'sunlight']
277 ['animals', 'need', 'food'] ['animals', 'need', 'food']
278 ['animals', 'need', 'a safe home'] ['animals', 'need', 'a safe home']
279 ['animals', 'need', 'oxygen'] ['anima

[['COUNTRY', '', 'HEMISPHERE (North, South, equatorial region)'],
 ['Angola', 'is located in the', 'southern hemisphere'],
 ['Botswana', 'is located in the', 'southern hemisphere'],
 ['Burundi', 'is located in the', 'southern hemisphere'],
 ['Comoros', 'is located in the', 'southern hemisphere'],
 ['Lesotho', 'is located in the', 'southern hemisphere'],
 ['Madagascar', 'is located in the', 'southern hemisphere'],
 ['Malawi', 'is located in the', 'southern hemisphere'],
 ['Mauritius', 'is located in the', 'southern hemisphere'],
 ['Samoa', 'is located in the', 'southern hemisphere'],
 ['Mozambique', 'is located in the', 'southern hemisphere'],
 ['Namibia', 'is located in the', 'southern hemisphere'],
 ['Rwanda', 'is located in the', 'southern hemisphere'],
 ['Seychelles', 'is located in the', 'southern hemisphere'],
 ['Swaziland', 'is located in the', 'southern hemisphere'],
 ['Tanzania', 'is located in the', 'southern hemisphere'],
 ['Zambia', 'is located in the', 'southern hemisphere'

In [ ]:
per_question_row_limit = 50
dev_percent = 20
test_percent = 20
question_over_row_limit_count = 0

In [ ]:
splits = []
for s in ['dev_lookup.jsonl', 'test_lookup.jsonl', 'train_lookup.jsonl']:
  splits.append(write_file(os.path.join("./",s)))

In [ ]:
with read_file("./MCQs.tsv") as fp:
  for i,parts in enumerate(csv.reader(fp,doublequote=False,delimiter='\t')):
    # print(i,parts)
    if i ==0:
      continue
    # print(len(parts))
    if (len(parts)!=10):
      print(f'bad line:{parts}')
      exit(1)

    # print(parts)
    qid = f'q{i}'
    qtext = parts[0].strip()
    quest_align = [int(c.strip()) for c in parts[1].split(',')]
    # print(quest_align)
    choices = [c.strip() for c in parts[2:6]]
    # print(choices)
    answer1 = choices[int(parts[6])-1]
    # print(answer)
    table_id = parts[7]
    target_row = int(parts[8])-1
    target_col = int(parts[9])
    # print(target_col)
    all_rows = tid2rows[table_id]
    # print(all_rows)
    header = all_rows[0]
    # print(header)
    rows = all_rows[1:]
    if target_col in quest_align:
      quest_align.remove(target_col)

    answer = rows[target_row][target_col]
    if(0<per_question_row_limit<len(rows)):
      pos_row = rows[target_row]
      neg_rows = rows[:target_row] + rows[target_row+1:]
      # print(len(neg_rows))
      rand.shuffle(neg_rows)
      rows = neg_rows[:per_question_row_limit]
      target_row = rand.randint(0,len(rows)-1)
      rows[target_row] = pos_row
      question_over_row_limit_count += 1
      # print(pos_row)


    # print(answer1," ----- ",answer2)
    data = {}
    data['id'] = qid
    data['question'] = qtext
    data['header'] = header
    data['rows'] = rows
    data['target_column'] = target_col
    data['answers'] = [answer1]
    data['table_id'] = table_id


    line = json.dumps(data) + '\n'

    if i % 100 < dev_percent:
      splits[0].write(line)
    elif i % 100 < dev_percent + test_percent:
      splits[1].write(line)
    else:
      splits[2].write(line)


for sfile in splits:
  sfile.close()


# print(f'{question_over_row_limit_count} questions over row limit')








In [ ]:
# tid2rows['monarch-67']

In [ ]:
from rank_bm25 import BM25Plus

ModuleNotFoundError: ignored

In [ ]:
def calculate_bm25(query, tables):

    query_tokens = query.split()
    table_tokens = [table["content"].split() for table in tables]
    bm25 = BM25Plus(table_tokens)
    scores = bm25.get_scores(query_tokens)
    table_scores = list(zip([table["table_id"] for table in tables], scores))
    table_scores.sort(key=lambda x: x[1], reverse=True)

    return table_scores


In [ ]:
with open("query.json", "r") as query_file:
  query_data = json.load(query_file)
with open("tables.json", "r") as tables_file:
    tables_data = json.load(tables_file)

# Extract query and tables from the JSON data
query = query_data["query"]
tables = tables_data["tables"]

# Calculate BM25 scores and get ranked tables
ranked_tables = calculate_bm25(query, tables)

# Print the table IDs and their corresponding BM25 scores
for table_id, score in ranked_tables:
    print(f"Table ID: {table_id}, BM25 Score: {score}")

In [ ]:
with read_file("WikiSQL_data/train_lookup.jsonl.gz") as fp:
  for i in fp:
    print(i)

FileNotFoundError: ignored

In [ ]:
a = read_file("train_lookup.jsonl.gz")

In [ ]:
path1 = "data_table_mcq.jsonl"
path2 = "data_wtq.jsonl"
path3 = "wiki_sql_data_lookup.jsonl"
path4 = "wiki_sql_data_agg.jsonl"


In [ ]:
! pip install rank_bm25

In [ ]:
import re
from rank_bm25 import BM25Okapi

In [ ]:
def pre_process(path):
  di = {}
  punc_pattern = r"[!\"#\$%&\'\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`{\|}~]"
  with read_file(path) as fp:

    for n1,line in enumerate(fp):
      data = json.loads(line)
      for k,v in data.items():
        qid = k
        header = v[0]
        rows = v[1:]
        # print(qid,header,rows)
        header1 = []
        for h in header:
          res = re.sub(punc_pattern,' ',h)
          res = re.sub("\s+",' ',res)
          header1.extend(res.lower().split())

        rows1 = []
        for i in rows:
          for j in i:
            res = re.sub(punc_pattern,' ',j)
            res = re.sub("\s+",' ',res)
            rows1.extend(res.lower().split())

        header1.extend(rows1)
        # print(header1)
        di[k] = header1
    return di


In [ ]:
def preprocess_query(query):
  punc_pattern = r"[!\"#\$%&\'\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`{\|}~]"
  res = re.sub(punc_pattern,' ',query)
  res = re.sub("\s+",' ',res)
  tokenized_query = res.lower().split()
  return tokenized_query

In [ ]:
query = "what is the temperature?"


In [ ]:
def ranking_docs(query,di):
  tokenized_query = preprocess_query(query)
  bm25 = BM25Okapi(di.values())
  scores = bm25.get_scores(tokenized_query)
  ranked_documents = dict(sorted(zip(di.keys(), scores), key=lambda x: x[1], reverse=True))
  return ranked_documents



In [ ]:
def BM25(query,top=300,paths=[path1,path2,path3]):
  di1 = pre_process(paths[0])
  di2 = pre_process(paths[1])
  di3 = pre_process(paths[2])
  ranked_doc1 = ranking_docs(query,di1)
  ranked_doc2 = ranking_docs(query,di2)
  ranked_doc3 = ranking_docs(query,di3)
  result = {**ranked_doc1,**ranked_doc2,**ranked_doc3}
  final_result = dict(list(sorted(result.items(), key=lambda x: x[1], reverse=True))[:top])

  tables = {}
  for i in paths:
    with read_file(i) as fp:
      for n1,line in enumerate(fp):
        data = json.loads(line)
        for k,v in data.items():
          if(k in final_result.keys()):
            tables[k] = v

  return tables

In [ ]:
tables = BM25(query)

In [ ]:
len(tables)

300

In [ ]:
with open('data_table_mcq.jsonl', 'w') as f:
    json.dump(tid2rows, f)

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

# Load the MobileBERT model
model = AutoModel.from_pretrained('google/mobilebert-uncased')

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('google/mobilebert-uncased')

# Prepare the input text
text = "Hello, world!"

# Tokenize the input text
encoded_text = tokenizer(text, return_tensors='pt')

# Get the embeddings from the last layer
with torch.no_grad():
    outputs = model(**encoded_text).pooler_output
    # embeddings = outputs[1].cpu().detach()

# Train another model using the embeddings
# ...



In [ ]:
outputs

tensor([[-2.5233e+07,  6.6200e+04,  1.8246e+05,  2.4427e+07,  3.4008e+07,
          2.9777e+05,  5.0000e+06,  6.9465e+06, -4.0925e+04,  6.7184e+03,
         -4.9095e+04, -2.0292e+04,  3.3900e+07,  3.3688e+07,  3.0054e+07,
         -1.1342e+07, -3.8427e+07, -1.0614e+07, -1.2340e+07, -1.4255e+07,
         -9.9282e+06,  1.5962e+07, -1.2762e+07, -3.7967e+06,  1.1085e+07,
         -1.8416e+07, -2.4860e+04,  2.4935e+07,  7.4520e+05,  1.7096e+05,
          9.6981e+06,  3.1108e+07, -4.9287e+04, -6.2170e+06,  9.7999e+06,
         -1.6464e+05, -1.5432e+05,  1.3773e+07, -3.2429e+07,  2.7026e+05,
         -3.6851e+04,  2.4280e+07, -6.2595e+05,  2.5508e+04, -3.7189e+07,
         -2.3039e+07,  4.7111e+07, -2.2208e+04, -3.5313e+07,  1.6481e+07,
         -3.3161e+07,  1.2323e+07,  1.1575e+05,  5.5629e+06, -3.5062e+07,
         -8.7658e+06,  9.5254e+05, -4.9227e+07,  1.1138e+07, -1.2051e+04,
          2.2019e+07, -4.0982e+06, -8.5656e+06, -1.2168e+07,  1.0272e+07,
         -2.8903e+05,  2.3363e+05, -1.

In [ ]:
embeddings

tensor([[-2.5233e+07,  6.6200e+04,  1.8246e+05,  2.4427e+07,  3.4008e+07,
          2.9777e+05,  5.0000e+06,  6.9465e+06, -4.0925e+04,  6.7184e+03,
         -4.9095e+04, -2.0292e+04,  3.3900e+07,  3.3688e+07,  3.0054e+07,
         -1.1342e+07, -3.8427e+07, -1.0614e+07, -1.2340e+07, -1.4255e+07,
         -9.9282e+06,  1.5962e+07, -1.2762e+07, -3.7967e+06,  1.1085e+07,
         -1.8416e+07, -2.4860e+04,  2.4935e+07,  7.4520e+05,  1.7096e+05,
          9.6981e+06,  3.1108e+07, -4.9287e+04, -6.2170e+06,  9.7999e+06,
         -1.6464e+05, -1.5432e+05,  1.3773e+07, -3.2429e+07,  2.7026e+05,
         -3.6851e+04,  2.4280e+07, -6.2595e+05,  2.5508e+04, -3.7189e+07,
         -2.3039e+07,  4.7111e+07, -2.2208e+04, -3.5313e+07,  1.6481e+07,
         -3.3161e+07,  1.2323e+07,  1.1575e+05,  5.5629e+06, -3.5062e+07,
         -8.7658e+06,  9.5254e+05, -4.9227e+07,  1.1138e+07, -1.2051e+04,
          2.2019e+07, -4.0982e+06, -8.5656e+06, -1.2168e+07,  1.0272e+07,
         -2.8903e+05,  2.3363e+05, -1.

In [ ]:
embeddings == outputs

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, T

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import MobileBertConfig, MobileBertForSequenceClassification

# albert_config = MobileBertConfig()
# model = MobileBertForSequenceClassification(albert_config)

# Load the MobileBERT model
model = AutoModel.from_pretrained('google/mobilebert-uncased')

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('google/mobilebert-uncased')

# Prepare the input text
text = [' : In the * HEMISPHERE : northern hemisphere *  : , the * ORBITAL EVENT : summer solstice *  : occurs in * MONTH OF OCCURENCE : June', ' : In the * HEMISPHERE : southern hemisphere *  : , the * ORBITAL EVENT : summer solstice *  : occurs in * MONTH OF OCCURENCE : December', ' : In the * HEMISPHERE : northern hemisphere *  : , the * ORBITAL EVENT : winter solstice *  : occurs in * MONTH OF OCCURENCE : December', ' : In the * HEMISPHERE : southern hemisphere *  : , the * ORBITAL EVENT : winter solstice *  : occurs in * MONTH OF OCCURENCE : June', ' : In the * HEMISPHERE : northern hemisphere *  : , the * ORBITAL EVENT : spring equinox *  : occurs in * MONTH OF OCCURENCE : March', ' : In the * HEMISPHERE : southern hemisphere *  : , the * ORBITAL EVENT : spring equinox *  : occurs in * MONTH OF OCCURENCE : September', ' : In the * HEMISPHERE : northern hemisphere *  : , the * ORBITAL EVENT : fall equinox *  : occurs in * MONTH OF OCCURENCE : September', ' : In the * HEMISPHERE : southern hemisphere *  : , the * ORBITAL EVENT : fall equinox *  : occurs in * MONTH OF OCCURENCE : March']

# Tokenize the input text
encoded_text = tokenizer(text, return_tensors='pt')
with torch.no_grad():
    outputs = model(**encoded_text).pooler_output

# Get the embeddings from the last layer
# with torch.no_grad():
#     # selected_batch = {'input_ids': encoded_text['input_ids'][:1, :], 'attention_mask': encoded_text['attention_mask'][:1, :]}
#     embeddings = model(encoded_text['input_ids'][:1, :], encoded_text['attention_mask'][:1, :])[1].cpu().detach()

# print(embeddings.shape)
print(encoded_text)

{'input_ids': tensor([[  101,  1024,  1999,  1996,  1008, 14130,  1024,  2642, 14130,  1008,
          1024,  1010,  1996,  1008, 13943,  2724,  1024,  2621, 14017, 10074,
          2063,  1008,  1024,  5158,  1999,  1008,  3204,  1997,  5258, 10127,
          1024,  2238,   102],
        [  101,  1024,  1999,  1996,  1008, 14130,  1024,  2670, 14130,  1008,
          1024,  1010,  1996,  1008, 13943,  2724,  1024,  2621, 14017, 10074,
          2063,  1008,  1024,  5158,  1999,  1008,  3204,  1997,  5258, 10127,
          1024,  2285,   102],
        [  101,  1024,  1999,  1996,  1008, 14130,  1024,  2642, 14130,  1008,
          1024,  1010,  1996,  1008, 13943,  2724,  1024,  3467, 14017, 10074,
          2063,  1008,  1024,  5158,  1999,  1008,  3204,  1997,  5258, 10127,
          1024,  2285,   102],
        [  101,  1024,  1999,  1996,  1008, 14130,  1024,  2670, 14130,  1008,
          1024,  1010,  1996,  1008, 13943,  2724,  1024,  3467, 14017, 10074,
          2063,  1008,  

In [ ]:
outputs[1] == outputs[2]

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, 